In [18]:
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt

In [37]:
def sRGB_to_linear(img):
    wasImage = False
    if isinstance(img, Image.Image):
        print("Dividing by 255")
        data = np.array(img).astype(float) / 255.
        wasImage = True
    else:
        data = img.astype(float) / 255.
    print(data)
    data = np.where(data <= 0.04045, data / 12.92, np.power((data + 0.055) / 1.055, 2.4))
    print(data)
    if wasImage:
        return Image.fromarray((data * 255.).astype(np.uint8))
    else:
        return (data * 255.).astype(np.uint8)


def linear_to_sRGB(img):
    wasImage = False
    if isinstance(img, Image.Image):
        data = np.array(img).astype(float) / 255.
        wasImage = True
    else:
        data = img.astype(float) / 255.
    data = np.where(data <= 0.0031308, data * 12.92, 1.055 * np.power(data, 1. / 2.4) - 0.055)
    if wasImage:
        return Image.fromarray((data * 255.).astype(np.uint8))
    else:
        return (data * 255.).astype(np.uint8)


def draw_debug_halo(img):
    nx, ny = img.size
    for yy in [0, ny-1]:
        for xx in range(nx):
            img.putpixel((xx, yy), (0, 255, 0, 255))
    for xx in [0, nx-1]:
        for yy in range(ny):
            img.putpixel((xx, yy), (0, 255, 0, 255))
    return img


def make_debug_square(N, col=(0, 255, 0, 255)):
    debug_square_arr = np.zeros((N, N, 4), dtype=np.uint8)
    for ic, c in enumerate(col):
        debug_square_arr[:, :, ic] = c
    return Image.fromarray(debug_square_arr)


def make_debug_cross(N, col=(0, 255, 0, 255)):
    debug_cross_arr = np.zeros((N, N, 4), dtype=np.uint8)
    t1 = int(N/2)
    t2 = int(N/2+1)
    for ic, c in enumerate(col):
        debug_cross_arr[t1:t2, :, ic] = c
        debug_cross_arr[:, t1:t2, ic] = c
    return Image.fromarray(debug_cross_arr)


def get_parent_directory():
    # full_path = __file__
    # return full_path.replace("app/strass_band_generator_v2.py", "")
    return "/home/phillip/python/strass_preview_generator/"


def download_img(urlstring, local_img_path):
    print("Downloading ", urlstring, " to ", local_img_path)
    urllib.request.urlretrieve(urlstring, local_img_path)
    time.sleep(1)
    print(os.path.isfile(local_img_path))
    img = sRGB_to_linear(Image.open(local_img_path))
    return img


def get_img(img_path, linear=True):
    parent_dir = get_parent_directory()
    local_img_path = parent_dir + img_path
    print("check for {:s}".format(local_img_path))
    OK = False
    if os.path.isfile(local_img_path):
        print("Found, opening {:s}".format(local_img_path))
        try:
            img = Image.open(local_img_path)
            img = sRGB_to_linear(img) if linear else img
            OK = True
        except:
            raise
            print("Opening {:s} failed, corrupt file? Falling back to download option.".format(local_img_path))
            OK = False
    if not OK or not img:
        print("{:s} not found, downloading.".format(local_img_path))
        img_url = "https://raw.githubusercontent.com/baeldin/strass_preview_generator/main/" + img_path
        img = download_img(img_url, local_img_path)
    return img, local_img_path

In [22]:
img, _ = get_img("models/Alcantara_045.png")

check for /home/phillip/python/strass_preview_generator/models/Alcantara_045.png
Found, opening /home/phillip/python/strass_preview_generator/models/Alcantara_045.png


In [45]:
imgs = Image.open("/home/phillip/python/strass_preview_generator/models/Alcantara_045.png")
imgl = sRGB_to_linear(imgs)
print(np.array(imgs))
print(np.array(imgl))
print(np.array(linear_to_sRGB(imgl)))


Dividing by 255
[[0.         0.00392157 0.00784314 ... 0.03921569 0.03529412 0.08235294]
 [0.21960784 0.         0.2        ... 0.08235294 0.38431373 0.03529412]
 [0.03921569 0.02745098 0.22352941 ... 0.36078431 0.10980392 0.10980392]
 ...
 [0.15294118 0.03529412 0.00392157 ... 0.00392157 0.14117647 0.03921569]
 [0.10196078 0.40784314 0.15294118 ... 0.03137255 0.15686275 0.15686275]
 [0.09019608 0.15294118 0.15294118 ... 0.1372549  0.08235294 0.03529412]]
[[0.         0.00030353 0.00060705 ... 0.00303527 0.00273174 0.00749903]
 [0.03954624 0.         0.03310477 ... 0.00749903 0.12213877 0.00273174]
 [0.00303527 0.00212469 0.0409152  ... 0.1070231  0.01161225 0.01161225]
 ...
 [0.02028856 0.00273174 0.00030353 ... 0.00030353 0.01764195 0.00303527]
 [0.01032982 0.13843162 0.02028856 ... 0.00242822 0.02121901 0.02121901]
 [0.00856813 0.02028856 0.02028856 ... 0.01680738 0.00749903 0.00273174]]
[[  0   1   2 ...  10   9  21]
 [ 56   0  51 ...  21  98   9]
 [ 10   7  57 ...  92  28  28]
 ..